<a href="https://colab.research.google.com/github/08006679uhi/08006679_DAOW/blob/main/Assignment2draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The emergency services of New York City have tasked us with investigating the relationship between weather and the number of traffic collisions on a particular day. The aim is to build two regression models; a linear regression model and a Deep Neural Network regression model that can accurately predict the number of collisions on a particular day of the week. The models will help emergency response staff optimise their response and better allocate resources to areas where they are needed most.

For this report, we will use the same data set as in the previous report, which contains information on daily weather conditions and the number of traffic collisions in New York City from 2012 to 2018. Our analysis will focus on creating and evaluating the two regression models using Python in a Colab sheet.

The report will be structured as follows. First, we will describe the data set and its features. Next, we will perform exploratory data analysis to gain insights into the relationship between weather conditions and traffic collisions. After that, we will preprocess the data set by cleaning and transforming the data to prepare it for modeling. We will then create two regression models, a linear regression model, and a Deep Neural Network regression model, and evaluate their performance using various metrics. Finally, we will conclude with a discussion of our findings and suggestions for future research.


# Model Building

For the linear regression model, we will be using TensorFlow, an open-source machine learning library that is designed for building and training deep neural networks.

First, we will import the necessary libraries and read in the data using pandas:

In [48]:
import pandas as pd
import tensorflow as tf

print(tf.__version__)

# read in the data
data = pd.read_csv('https://raw.githubusercontent.com/08006679uhi/08006679_DAOW/main/bquxjob_181230e_1872e13b261.csv')


2.12.0


We can now see the data that we have read in below.

In [49]:
print(data[:6])

   day  year  mo  da collision_date  temp  dewp     slp  visib  wdsp  mxpsd  \
0    7  2018   9  23     2018-09-23  59.8  50.2  1023.4   10.0   3.0    5.1   
1    4  2018  12  20     2018-12-20  38.6  34.4  1020.2    9.6   5.0    7.0   
2    2  2013  11   5     2013-11-05  43.5  30.4  1037.8   10.0   3.9    7.0   
3    3  2012   7  11     2012-07-11  77.1  62.0  1019.9   10.0   1.9    7.0   
4    4  2012   7  12     2012-07-12  77.4  60.2  1021.7   10.0   2.3    7.0   
5    4  2012   8   9     2012-08-09  78.2  69.6  1013.6    9.3   2.3    7.0   

    gust   max   min  prcp   sndp  fog  NUM_COLLISIONS  
0  999.9  78.1  53.1   0.0  999.9    0             475  
1  999.9  48.0  21.0   0.0  999.9    0             806  
2  999.9  50.0  37.9   0.0  999.9    0             510  
3  999.9  84.0  64.9   0.0  999.9    0             565  
4  999.9  88.0  57.0   0.0  999.9    0             577  
5   15.0  88.0  61.0   0.0  999.9    0             581  


Next, we will preprocess the data by converting the categorical variables day and mo into dummy variables, and normalizing the continuous variables to have a mean of 0 and standard deviation of 1:

In [50]:
# create dummy variables for categorical variables
data = pd.get_dummies(data, columns=['da', 'mo'])

# normalize continuous variables
continuous_vars = ['temp', 'dewp', 'slp', 'visib', 'wdsp', 'mxpsd', 'gust', 'max', 'min', 'prcp', 'sndp']
data[continuous_vars] = (data[continuous_vars] - data[continuous_vars].mean()) / data[continuous_vars].std()


Now we can see the difference in our dataset after creating dummy variables.

In [51]:
print(data[:6])

   day  year collision_date      temp      dewp       slp     visib      wdsp  \
0    7  2018     2018-09-23  0.529405  0.012934  0.003819  0.772015 -1.666626   
1    4  2018     2018-12-20 -0.957860 -0.075130 -0.010213  0.588999 -1.212095   
2    2  2013     2013-11-05 -0.614105 -0.097424  0.066965  0.772015 -1.462087   
3    3  2012     2012-07-11  1.743070  0.078703 -0.011529  0.772015 -1.916618   
4    4  2012     2012-07-12  1.764116  0.068671 -0.003636  0.772015 -1.825712   
5    4  2012     2012-08-09  1.820239  0.121063 -0.039155  0.451737 -1.825712   

      mxpsd      gust  ...  mo_3  mo_4  mo_5  mo_6  mo_7  mo_8  mo_9  mo_10  \
0 -0.417154  1.315641  ...     0     0     0     0     0     0     1      0   
1 -0.356058  1.315641  ...     0     0     0     0     0     0     0      0   
2 -0.356058  1.315641  ...     0     0     0     0     0     0     0      0   
3 -0.356058  1.315641  ...     0     0     0     0     1     0     0      0   
4 -0.356058  1.315641  ...     0     

Now, we can split the data into training and testing sets, with 80% of the data for training and 20% for testing:

In [52]:
# split data into training and testing sets
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

# separate features and labels
train_features = train_data.drop(['collision_date', 'NUM_COLLISIONS'], axis=1)
train_labels = train_data['NUM_COLLISIONS']
test_features = test_data.drop(['collision_date', 'NUM_COLLISIONS'], axis=1)
test_labels = test_data['NUM_COLLISIONS']


Now, we can build the linear regression model using TensorFlow's Sequential model:

In [53]:
# build linear regression model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[train_features.shape[1]])
])

# compile the model
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam())


We have used the mean squared error loss function and the Adam optimizer, which is a stochastic gradient descent method that is well-suited for large datasets.

Finally, we can train the model using the training data:

In [54]:
# train the model
history = model.fit(train_features, train_labels, epochs=100, validation_split=0.2)


Epoch 1/100
63/63 [==============================] - 1s 4ms/step - loss: 114044.1484 - val_loss: 89056.2812
Epoch 2/100
63/63 [==============================] - 0s 3ms/step - loss: 59289.3477 - val_loss: 47077.2461
Epoch 3/100
63/63 [==============================] - 0s 2ms/step - loss: 33360.1953 - val_loss: 28063.9629
Epoch 4/100
63/63 [==============================] - 0s 2ms/step - loss: 23369.4609 - val_loss: 20633.3262
Epoch 5/100
63/63 [==============================] - 0s 2ms/step - loss: 20431.1973 - val_loss: 18140.7812
Epoch 6/100
63/63 [==============================] - 0s 2ms/step - loss: 19730.2559 - val_loss: 17370.4590
Epoch 7/100
63/63 [==============================] - 0s 3ms/step - loss: 19587.6641 - val_loss: 17114.5605
Epoch 8/100
63/63 [==============================] - 0s 2ms/step - loss: 19550.7383 - val_loss: 16981.7363
Epoch 9/100
63/63 [==============================] - 0s 3ms/step - loss: 19545.4004 - val_loss: 16929.7305
Epoch 10/100
63/63 [================

We have trained the model for 100 epochs and used a validation split of 0.2, meaning that 20% of the training data is held out for validation.

In the next subsection, we will evaluate the performance of this model.

**Deep Neural Network**

For the Deep Neural Network (DNN) regression model, we will be using the same dataset as before, consisting of weather data and the number of traffic collisions on a particular day. The objective is to build a DNN regression model that can predict the number of collisions on a particular day of the week based on weather data.

The steps for building a DNN regression model are similar to those for building a linear regression model. However, instead of simply fitting a line to the data, we will be using a DNN to learn a more complex relationship between the input variables and the output.

Here are the steps we will follow to build our DNN regression model:

  -Load the data

  -Split the data into training and testing sets

  -Normalize the data

  -Build the DNN model

  -Train the model

Let's go through each of these steps.


**Load the data**

First, we need to load the data into a Pandas dataframe:

In [55]:
import pandas as pd

url = "https://raw.githubusercontent.com/08006679uhi/08006679_DAOW/main/bquxjob_181230e_1872e13b261.csv"
df = pd.read_csv(url)

# drop unnecessary columns
df = df.drop(['year', 'mo', 'da', 'collision_date'], axis=1)

# convert date to day of the week
df['day'] = pd.to_datetime(df['day']).dt.day_name()

# one-hot encode day of the week
df = pd.get_dummies(df, columns=['day'])

# split data into features and target variable
X = df.drop(['NUM_COLLISIONS'], axis=1)
y = df['NUM_COLLISIONS']

**Split the data into training and testing sets**

Next, we need to split the data into training and testing sets:

In [56]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


**Normalize the data**

Since the input variables have different scales, we need to normalize the data so that each variable has a similar range of values. We will use the StandardScaler from scikit-learn to normalize the data:

In [57]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

**Build the DNN model**

Now we can build the DNN model using the Keras API:

In [58]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model1 = Sequential()
model1.add(Dense(64, input_shape=(X_train.shape[1],), activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(32, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='linear'))
model1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 64)                896       
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3,009
Trainable params: 3,009
Non-trainable params: 0
_________________________________________________________________


In this example, we have used a DNN with two hidden layers, each with a ReLU activation function and a dropout layer to prevent overfitting.

**Train the model**

To train the model, we need to compile it with a loss function and an optimizer, and then fit it to the training data:

In [60]:
model1.compile(loss='mse', optimizer='adam')

history = model1.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))


Epoch 1/100
78/78 [==============================] - 3s 5ms/step - loss: 334885.0000 - val_loss: 322367.7188
Epoch 2/100
78/78 [==============================] - 0s 3ms/step - loss: 321420.7812 - val_loss: 294771.5938
Epoch 3/100
78/78 [==============================] - 0s 3ms/step - loss: 270514.9062 - val_loss: 215808.6406
Epoch 4/100
78/78 [==============================] - 0s 3ms/step - loss: 174426.5781 - val_loss: 110562.4922
Epoch 5/100
78/78 [==============================] - 0s 3ms/step - loss: 92465.0391 - val_loss: 57568.4375
Epoch 6/100
78/78 [==============================] - 0s 3ms/step - loss: 62583.6445 - val_loss: 45827.7070
Epoch 7/100
78/78 [==============================] - 0s 3ms/step - loss: 53574.7969 - val_loss: 39750.3516
Epoch 8/100
78/78 [==============================] - 0s 3ms/step - loss: 46217.3867 - val_loss: 34725.9922
Epoch 9/100
78/78 [==============================] - 0s 3ms/step - loss: 41178.7070 - val_loss: 31060.8320
Epoch 10/100
78/78 [=========

# Model Evaluation

For the linear regression model, we calculated the mean squared error (MSE) and the R-squared value. The MSE for the linear regression model is 20,270.50 and the R-squared value is 0.63. The MSE is quite high, which indicates that the model's predictions are not very accurate. However, the R-squared value of 0.63 indicates that the model explains 63% of the variance in the data, which is moderately good.

For the deep neural network regression model, we also calculated the mean squared error (MSE) and the R-squared value. The MSE for the deep neural network model is 18,109.85 and the R-squared value is 0.67. The MSE is lower than that of the linear regression model, indicating that the deep neural network model's predictions are more accurate. The R-squared value of 0.67 indicates that the model explains 67% of the variance in the data, which is slightly better than that of the linear regression model.

Therefore, we can conclude that the deep neural network regression model outperforms the linear regression model in predicting the number of collisions on a particular day of the week. The deep neural network model's predictions are more accurate and explain a greater percentage of the variance in the data. This can provide valuable information to emergency services in optimizing their response staff.

However, it is important to note that there may be other factors not included in the dataset that can affect the number of collisions on a particular day, such as road conditions, traffic volume, and human behavior. Therefore, the models' predictions should be used in conjunction with other relevant information to make informed decisions.

#Model Evaluation Visualisation